In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
csv_path = os.path.join('..','datasets','raw','data.csv')
df = pd.read_csv(csv_path)
df.sample(10)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
8345,Ford,Ranger,2009,regular unleaded,143.0,4.0,MANUAL,rear wheel drive,2.0,NaN,Compact,Extended Cab Pickup,26,21,5657,19080
5463,Volkswagen,Golf,2016,regular unleaded,170.0,4.0,MANUAL,front wheel drive,4.0,Hatchback,Compact,4dr Hatchback,37,25,873,20175
498,Toyota,4Runner,2016,regular unleaded,270.0,6.0,AUTOMATIC,four wheel drive,4.0,NaN,Midsize,4dr SUV,21,17,2031,39595
3745,Ford,E-Series Wagon,2013,flex-fuel (unleaded/E85),255.0,8.0,AUTOMATIC,rear wheel drive,3.0,Flex Fuel,Midsize,Passenger Van,15,11,5657,36545
10278,Ford,Taurus X,2009,regular unleaded,263.0,6.0,AUTOMATIC,front wheel drive,4.0,Crossover,Large,Wagon,24,17,5657,32325
761,BMW,8 Series,1995,regular unleaded,372.0,12.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,Performance",Midsize,Coupe,18,10,3916,8488
3405,Mazda,CX-9,2014,regular unleaded,273.0,6.0,AUTOMATIC,all wheel drive,4.0,Crossover,Large,4dr SUV,22,16,586,34070
2549,Honda,Civic,2015,regular unleaded,143.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,36,28,2202,20390
8401,Toyota,RAV4,2017,regular unleaded,176.0,4.0,AUTOMATIC,front wheel drive,4.0,Crossover,Midsize,4dr SUV,29,23,2031,34750
8408,Lexus,RC 300,2017,premium unleaded (required),255.0,6.0,AUTOMATIC,all wheel drive,2.0,Luxury,Midsize,Coupe,26,19,454,42770


In [3]:
[col for col in df.columns]

['Make',
 'Model',
 'Year',
 'Engine Fuel Type',
 'Engine HP',
 'Engine Cylinders',
 'Transmission Type',
 'Driven_Wheels',
 'Number of Doors',
 'Market Category',
 'Vehicle Size',
 'Vehicle Style',
 'highway MPG',
 'city mpg',
 'Popularity',
 'MSRP']

In [4]:
my_variables = ['Make','Model','Year','Engine HP','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg','MSRP']
cars = df[my_variables].copy()
cars.sample(5)

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,MSRP
2985,Chevrolet,Corvette,2015,455.0,8.0,MANUAL,Coupe,29,17,59160
7600,Honda,Pilot,2015,250.0,6.0,AUTOMATIC,4dr SUV,25,18,35370
7479,Volkswagen,Passat,2016,280.0,6.0,AUTOMATED_MANUAL,Sedan,28,20,36835
1755,Subaru,B9 Tribeca,2006,250.0,6.0,AUTOMATIC,4dr SUV,21,16,35895
2978,Chevrolet,Corvette,2015,460.0,8.0,MANUAL,Coupe,29,17,60000


In [5]:
# Lowercase the column names and replace spaces with underscores
cars.columns = cars.columns.str.lower().str.replace(' ','_')

In [6]:
# View missing values
cars.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [7]:
# FIll the missing values with 0
cars[['engine_hp','engine_cylinders']] = cars[['engine_hp','engine_cylinders']].fillna(value=0)

In [8]:
msrp_mean = cars['msrp'].mean()
bin_msrp = [0, msrp_mean, float('inf')]
cars['msrp_binarized'] = pd.cut(cars['msrp'], bins=bin_msrp, labels=[1,0])
cars['msrp_binarized']

0        0
1        0
2        1
3        1
4        1
        ..
11909    0
11910    0
11911    0
11912    0
11913    1
Name: msrp_binarized, Length: 11914, dtype: category
Categories (2, int64): [1 < 0]

In [13]:
features = cars.drop('msrp', axis=1).copy()
target = cars['msrp'].copy()

X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=.5, random_state=42)
X_train.shape[0], X_test.shape[0], X_val.shape[0]

(4765, 3574, 3575)